<a href="https://colab.research.google.com/github/Jerome-Joseph-1/Merging_Bioactivity_Predictions_CellPainting_Chemical_Structure_Similarity/blob/develop/bionet_actual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
# !pip install -U scikit-learn tqdm
# !pip install torch

In [62]:
import pandas as pd
import numpy as np

In [63]:
df1 = pd.read_csv('/content/drive/MyDrive/Bioactivity_Nerual_Network/Analysis/Datasets/merged_assay_details_with_ensemble.csv').drop(['Unnamed: 0'], axis = 1)

In [64]:
df1

,assay,index,InChICode_standardised,predicted_value_fp,final_value_fp,expected_value_fp,match_fp,predicted_value_cp,final_value_cp,expected_value_cp,...,pc_actives,pc_perc_actives,pc_inactives,pc_perc_inactives,Data,CP_Best_Threshold,FP_Best_Threshold,predicted_value_fp_scaled,predicted_value_cp_scaled,simple_pred
0,588458,313,InChI=1S/C14H15BrN2O3/c1-17(2)6-5-16-13(18)11-...,0.436863,1,0.0,False,0.480519,1,0.0,...,0.311134,0.000000,0.325709,0.000000,HeldOut,0.236092,0.289240,0.603849,0.659985,0.631917
1,588458,113,InChI=1S/C23H22N2O2/c26-23(16-22-21-9-5-4-6-17...,0.160961,0,0.0,True,0.361142,1,0.0,...,0.627549,0.088889,0.698505,0.164179,HeldOut,0.236092,0.289240,0.278248,0.581849,0.430048
2,588458,218,InChI=1S/C22H17F2NO3/c23-18-7-1-15(2-8-18)13-2...,0.293416,1,0.0,False,0.118246,0,0.0,...,0.420571,0.011111,0.751601,0.022388,HeldOut,0.236092,0.289240,0.502938,0.250422,0.376680
3,588458,371,InChI=1S/C24H24N6O4/c25-22-20(24(31)26-7-8-29-...,0.158796,0,0.0,True,0.312089,1,0.0,...,0.585947,0.100000,0.630570,0.100746,HeldOut,0.236092,0.289240,0.274505,0.549742,0.412124
4,588458,200,InChI=1S/C15H23BrN2O4S/c1-10-8-18(11(2)9-19)23...,0.633254,1,0.0,False,0.529011,1,0.0,...,0.772584,0.155556,0.756990,0.063433,HeldOut,0.236092,0.289240,0.742005,0.691724,0.716864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10067,1117305,2752,"InChI=1S/C7H8ClN3O4S2/c8-4-1-5-7(2-6(4)16(9,12...",0.384577,0,0.0,True,0.413604,1,0.0,...,0.605005,0.111111,0.628868,0.055838,HeldOut,0.362270,0.409406,0.469678,0.540247,0.504962
10068,1117305,2894,InChI=1S/C14H9I3O4/c15-9-6-8(1-2-12(9)18)21-14...,0.322409,0,0.0,True,0.419348,1,0.0,...,0.532951,0.044444,0.584091,0.060914,HeldOut,0.362270,0.409406,0.393752,0.544751,0.469252
10069,1117305,958,InChI=1S/C18H19Cl2NO4/c1-5-25-18(23)14-10(3)21...,0.223534,0,1.0,False,0.386036,1,1.0,...,0.685987,0.244444,0.723294,0.167513,HeldOut,0.362270,0.409406,0.272998,0.518633,0.395815
10070,1117305,2067,InChI=1S/C16H11BrN2O/c17-9-5-6-14-11(7-9)12-8-...,0.418716,1,0.0,False,0.257775,0,0.0,...,0.697407,0.100000,0.812100,0.142132,HeldOut,0.362270,0.409406,0.507883,0.355778,0.431830


In [65]:
features = ['assay', 'expected_value_fp', 'predicted_value_fp_scaled','predicted_value_cp_scaled', 'ts_actives', 'pc_actives', 'InChICode_standardised']

In [66]:
# def test2():
#   import torch
#   import torch.nn as nn
#   import torch.optim as optim
#   from torch.utils.data import DataLoader, TensorDataset
#   from sklearn.preprocessing import StandardScaler
#   from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, roc_curve
#   import pandas as pd
#   import numpy as np
#   from tqdm import tqdm
#   from sklearn.model_selection import train_test_split

#   # Load data
#   data = df1[features]

#   # Results and predictions storage
#   results = []
#   all_predictions = []

#   # Define parameter grid
#   param_grid = {
#       'learning_rate': [0.001, 0.01],
#       'batch_size': [32, 64],
#       'hidden_units': [(16, 8), (32, 16)]
#   }

#   # Define the neural network architecture dynamically
#   class BinaryClassifier(nn.Module):
#       def __init__(self, hidden_units):
#           super(BinaryClassifier, self).__init__()
#           self.layer1 = nn.Linear(4, hidden_units[0])
#           self.layer2 = nn.Linear(hidden_units[0], hidden_units[1])
#           self.output = nn.Linear(hidden_units[1], 1)
#           self.relu = nn.ReLU()

#       def forward(self, x):
#           x = self.relu(self.layer1(x))
#           x = self.relu(self.layer2(x))
#           return torch.sigmoid(self.output(x))

#   # Training and evaluation per assay with parameter tuning
#   for assay in tqdm(data['assay'].unique()):
#       assay_data = data[data['assay'] == assay]
#       X = assay_data.drop(['expected_value_fp', 'assay', 'InChICode_standardised'], axis=1)
#       y = assay_data['expected_value_fp']
#       compounds = assay_data['InChICode_standardised']  # Assumed compound identifier column

#       X_train, X_test, y_train, y_test, compounds_train, compounds_test = train_test_split(X, y, compounds, test_size=0.2, random_state=42)

#       best_auc = 0
#       best_params = {}
#       best_threshold = 0

#       # Parameter tuning
#       for lr in param_grid['learning_rate']:
#           for batch_size in param_grid['batch_size']:
#               for hidden_units in param_grid['hidden_units']:
#                   # Scaling
#                   scaler = StandardScaler()
#                   X_train_scaled = scaler.fit_transform(X_train)
#                   X_test_scaled = scaler.transform(X_test)

#                   # Convert to tensors
#                   X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
#                   y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.float32).view(-1, 1)
#                   X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
#                   y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.float32).view(-1, 1)

#                   # Load data into DataLoader
#                   train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)

#                   # Initialize the model
#                   model = BinaryClassifier(hidden_units)
#                   criterion = nn.BCELoss()
#                   optimizer = optim.Adam(model.parameters(), lr=lr)

#                   # Train the model
#                   num_epochs = 50
#                   for epoch in range(num_epochs):
#                       model.train()
#                       for inputs, labels in train_loader:
#                           optimizer.zero_grad()
#                           outputs = model(inputs)
#                           loss = criterion(outputs, labels)
#                           loss.backward()
#                           optimizer.step()

#                   # Validate the model
#                   model.eval()
#                   with torch.no_grad():
#                       y_pred = model(X_test_tensor).numpy()
#                       auc = roc_auc_score(y_test, y_pred)
#                       if auc > best_auc:
#                           best_auc = auc
#                           best_params = {'learning_rate': lr, 'batch_size': batch_size, 'hidden_units': hidden_units, 'auc': auc}
#                           best_model = model
#                           fpr, tpr, thresholds = roc_curve(y_test, y_pred)
#                           j_scores = tpr - fpr
#                           best_threshold = thresholds[np.argmax(j_scores)]

#       # Evaluate the best model
#       best_model.eval()
#       with torch.no_grad():
#           final_pred = best_model(X_test_tensor).numpy()
#           best_auc = roc_auc_score(y_test, final_pred)
#           final_pred_classes = (final_pred >= best_threshold).astype(int)
#           precision = precision_score(y_test, final_pred_classes)
#           recall = recall_score(y_test, final_pred_classes)
#           f1 = f1_score(y_test, final_pred_classes)
#           for comp, pred, actual in zip(compounds_test, final_pred, y_test):
#               all_predictions.append({
#                   'Compound': comp,
#                   'Predicted Probability': pred[0],
#                   'Actual Label': actual,
#                   'Assay': assay
#               })

#       results.append({
#           'Assay': assay,
#           'Best AUC': best_auc,
#           'Best Parameters': best_params,
#           'Best Threshold': best_threshold,
#           'Precision': precision,
#           'Recall': recall,
#           'F1 Score': f1
#       })

#   # Save results and predictions to CSV
#   results_df = pd.DataFrame(results)
#   results_df.to_csv('assay_metrics_with_tuning_6.csv', index=False)

#   predictions_df = pd.DataFrame(all_predictions)
#   predictions_df.to_csv('predictions_per_compound_6.csv', index=False)

#   # Calculate the mean AUC across all assays
#   overall_auc_mean = np.mean([result['Best AUC'] for result in results])
#   pd.DataFrame({'Overall Mean AUC': [overall_auc_mean]}).to_csv('overall_mean_auc_6.csv', index=False)
# test2()


In [67]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, roc_curve
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Load data
data = df1[features]

# Results and predictions storage
results = []
all_predictions = []

# Define parameter grid
param_grid = {
    'learning_rate': [0.001, 0.01],
    'batch_size': [32, 64],
    'hidden_units': [(16, 8), (32, 16)]
}

# Define the neural network architecture dynamically
class BinaryClassifier(nn.Module):
    def __init__(self, hidden_units):
        super(BinaryClassifier, self).__init__()
        self.layer1 = nn.Linear(4, hidden_units[0])
        self.layer2 = nn.Linear(hidden_units[0], hidden_units[1])
        self.output = nn.Linear(hidden_units[1], 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = torch.sigmoid(self.output(x))
        return x

# Training and evaluation per assay with parameter tuning
for assay in tqdm(data['assay'].unique()):
    assay_data = data[data['assay'] == assay]
    X = assay_data.drop(['expected_value_fp', 'assay', 'InChICode_standardised'], axis=1)
    y = assay_data['expected_value_fp']
    compounds = assay_data['InChICode_standardised']  # Assumed compound identifier column

    X_train, X_test, y_train, y_test, compounds_train, compounds_test = train_test_split(X, y, compounds, test_size=0.2, random_state=42)

    best_auc_training = 0
    best_params = {}
    best_threshold = 0

    # Parameter tuning
    for lr in param_grid['learning_rate']:
        for batch_size in param_grid['batch_size']:
            for hidden_units in param_grid['hidden_units']:
                # Scaling
                scaler = StandardScaler()
                X_train_scaled = scaler.fit_transform(X_train)
                X_test_scaled = scaler.transform(X_test)

                # Convert to tensors
                X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
                y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.float32).view(-1, 1)
                X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
                y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.float32).view(-1, 1)

                # Load data into DataLoader
                train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)

                # Initialize the model
                model = BinaryClassifier(hidden_units)
                criterion = nn.BCELoss()
                optimizer = optim.Adam(model.parameters(), lr=lr)

                # Train the model
                num_epochs = 50
                for epoch in range(num_epochs):
                    for inputs, labels in train_loader:
                        optimizer.zero_grad()
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        loss.backward()
                        optimizer.step()

                # Validate the model
                model.eval()
                with torch.no_grad():
                    y_pred = model(X_test_tensor).numpy()
                    auc = roc_auc_score(y_test, y_pred)

                    if auc > best_auc_training:
                        best_auc_training = auc
                        best_params = {'learning_rate': lr, 'batch_size': batch_size, 'hidden_units': hidden_units, 'auc': auc}
                        best_model = model
                        # Calculate the best threshold using Youden's J statistic
                        fpr, tpr, thresholds = roc_curve(y_test, y_pred)
                        j_scores = tpr - fpr
                        best_threshold = thresholds[np.argmax(j_scores)]

    # Save the model with the best parameters
    torch.save(best_model.state_dict(), f'/content/drive/MyDrive/Bioactivity_Nerual_Network/Analysis/{assay}_best_model_state_dict.pth')

    # Evaluate the model with the best parameters and threshold
    best_model.eval()
    with torch.no_grad():
        final_pred = best_model(X_test_tensor).numpy()
        best_auc_testing = roc_auc_score(y_test, final_pred)
        final_pred_classes = (final_pred >= best_threshold).astype(int)
        precision = precision_score(y_test, final_pred_classes)
        recall = recall_score(y_test, final_pred_classes)
        f1 = f1_score(y_test, final_pred_classes)
        for comp, pred, actual in zip(compounds_test, final_pred, y_test):
            all_predictions.append({
                'Compound': comp,
                'Predicted Probability': pred[0],
                'Actual Label': actual,
                'Assay': assay
            })

    results.append({
        'Assay': assay,
        'Best AUC (Testing)': best_auc_testing,
        'Best Parameters': best_params,
        'Best Threshold': best_threshold,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    })


# Save results and predictions to CSV
results_df = pd.DataFrame(results)
results_df.to_csv('assay_metrics_with_tuning_7.csv', index=False)

predictions_df = pd.DataFrame(all_predictions)
predictions_df.to_csv('predictions_per_compound_7.csv', index=False)

# Calculate the mean AUC across all assays
overall_auc_mean = np.mean([result['Best AUC (Testing)'] for result in results])
pd.DataFrame({'Overall Mean AUC (Testing)': [overall_auc_mean]}).to_csv('overall_mean_auc_7.csv', index=False)


100%|██████████| 88/88 [02:31<00:00,  1.72s/it]


In [68]:
overall_auc_mean

0.7123699808225257

In [69]:
results_df

,Assay,Best AUC (Testing),Best Parameters,Best Threshold,Precision,Recall,F1 Score
0,588458,0.692308,"{'learning_rate': 0.001, 'batch_size': 32, 'hi...",0.258484,0.500000,1.000000,0.666667
1,588334,0.902778,"{'learning_rate': 0.01, 'batch_size': 64, 'hid...",0.306224,0.500000,1.000000,0.666667
2,2642,0.608696,"{'learning_rate': 0.001, 'batch_size': 64, 'hi...",0.487094,1.000000,0.375000,0.545455
3,2156,0.794118,"{'learning_rate': 0.001, 'batch_size': 32, 'hi...",0.293323,0.555556,0.833333,0.666667
4,2330,0.862500,"{'learning_rate': 0.01, 'batch_size': 64, 'hid...",0.999878,1.000000,0.750000,0.857143
...,...,...,...,...,...,...,...
83,720504,0.750000,"{'learning_rate': 0.001, 'batch_size': 64, 'hi...",0.635771,1.000000,0.750000,0.857143
84,720532,0.666667,"{'learning_rate': 0.01, 'batch_size': 32, 'hid...",0.999963,1.000000,0.500000,0.666667
85,1159524,0.741935,"{'learning_rate': 0.01, 'batch_size': 32, 'hid...",0.082908,0.250000,1.000000,0.400000
86,1117304,0.527273,"{'learning_rate': 0.01, 'batch_size': 32, 'hid...",0.020179,0.384615,1.000000,0.555556


In [70]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
# import pandas as pd
# import numpy as np
# from tqdm import tqdm

# # Load data
# data = df1[features]

# # Metrics storage
# results = []

# # Define parameter grid
# param_grid = {
#     'learning_rate': [0.001, 0.01],
#     'batch_size': [32, 64],
#     'hidden_units': [(16, 8), (32, 16)]
# }

# # Define the neural network architecture dynamically
# class BinaryClassifier(nn.Module):
#     def __init__(self, hidden_units):
#         super(BinaryClassifier, self).__init__()
#         self.layer1 = nn.Linear(4, hidden_units[0])
#         self.layer2 = nn.Linear(hidden_units[0], hidden_units[1])
#         self.output = nn.Linear(hidden_units[1], 1)
#         self.relu = nn.ReLU()

#     def forward(self, x):
#         x = self.relu(self.layer1(x))
#         x = self.relu(self.layer2(x))
#         x = torch.sigmoid(self.output(x))
#         return x

# # Training and evaluation per assay with parameter tuning
# for assay in tqdm(data['assay'].unique()):
#     assay_data = data[data['assay'] == assay]
#     X = assay_data.drop(['expected_value_fp', 'assay', 'InChICode_standardised'], axis=1)
#     y = assay_data['expected_value_fp']

#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#     best_auc = 0
#     best_params = {}

#     # Parameter tuning
#     for lr in param_grid['learning_rate']:
#         for batch_size in param_grid['batch_size']:
#             for hidden_units in param_grid['hidden_units']:
#                 # Scaling
#                 scaler = StandardScaler()
#                 X_train_scaled = scaler.fit_transform(X_train)
#                 X_test_scaled = scaler.transform(X_test)

#                 # Convert to tensors
#                 X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
#                 y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.float32).view(-1, 1)
#                 X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
#                 y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.float32).view(-1, 1)

#                 # Load data into DataLoader
#                 train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)

#                 # Initialize the model
#                 model = BinaryClassifier(hidden_units)
#                 criterion = nn.BCELoss()
#                 optimizer = optim.Adam(model.parameters(), lr=lr)

#                 # Train the model
#                 num_epochs = 50
#                 for epoch in range(num_epochs):
#                     for inputs, labels in train_loader:
#                         optimizer.zero_grad()
#                         outputs = model(inputs)
#                         loss = criterion(outputs, labels)
#                         loss.backward()
#                         optimizer.step()

#                 # Validate the model
#                 model.eval()
#                 with torch.no_grad():
#                     y_pred = model(X_test_tensor)
#                     auc = roc_auc_score(y_test_tensor.numpy(), y_pred.numpy())

#                     if auc > best_auc:
#                         best_auc = auc
#                         best_params = {'learning_rate': lr, 'batch_size': batch_size, 'hidden_units': hidden_units, 'auc': auc}

#     # Evaluate the model with the best parameters
#     results.append({
#         'Assay': assay,
#         'Best AUC': best_auc,
#         'Best Parameters': best_params
#     })

# # Save results to CSV
# results_df = pd.DataFrame(results)
# results_df.to_csv('assay_metrics_with_tuning_2.csv', index=False)

# # Calculate the mean AUC across all assays
# overall_auc_mean = np.mean([result['Best AUC'] for result in results])
# pd.DataFrame({'Overall Mean AUC': [overall_auc_mean]}).to_csv('overall_mean_auc_2.csv', index=False)


In [71]:
# # prompt: read the file inside the datasets folder inside bioactivity Analysis

# import pandas as pd

# # Define the path to the file
# file_path_1 = '/content/drive/MyDrive/Bioactivity_Nerual_Network/Analysis/Datasets/assay_metrics_with_tuning_5.csv'
# file_path_2 = '/content/drive/MyDrive/Bioactivity_Nerual_Network/Analysis/Datasets/overall_mean_auc_5.csv'

# # Read the file into a Pandas DataFrame
# df_1 = pd.read_csv(file_path_1)
# # df_2 = pd.read_csv(file_path_2)

# # Print the DataFrame
# # print(df_1)
# print(df_1['Best Parameters'])


In [72]:
# predictions_df

In [73]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, roc_curve
# import pandas as pd
# import numpy as np
# from tqdm import tqdm

# # Load data
# data = df1[features]

# # Results and predictions storage
# results = []
# all_predictions = []

# # Define the model architecture dynamically
# class BinaryClassifier(nn.Module):
#     def __init__(self, hidden_units):
#         super(BinaryClassifier, self).__init__()
#         self.layer1 = nn.Linear(4, hidden_units[0])
#         self.layer2 = nn.Linear(hidden_units[0], hidden_units[1])
#         self.output = nn.Linear(hidden_units[1], 1)
#         self.relu = nn.ReLU()

#     def forward(self, x):
#         x = self.relu(self.layer1(x))
#         x = self.relu(self.layer2(x))
#         x = torch.sigmoid(self.output(x))
#         return x

# # Predefined parameters
# params = {
#     'learning_rate': 0.001,
#     'batch_size': 64,
#     'hidden_units': (32, 16)
# }

# # Evaluation per assay
# for assay in tqdm(data['assay'].unique()):
#     assay_data = data[data['assay'] == assay]
#     X = assay_data.drop(['expected_value_fp', 'assay', 'InChICode_standardised'], axis=1)
#     y = assay_data['expected_value_fp']
#     compounds = assay_data['InChICode_standardised']  # Assumed compound identifier column
#     print(assay_data)
#     # Scaling
#     scaler = StandardScaler()
#     X_scaled = scaler.fit_transform(X)

#     # Convert to tensors
#     X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
#     y_tensor = torch.tensor(y.to_numpy(), dtype=torch.float32).view(-1, 1)

#     # Load data into DataLoader
#     loader = DataLoader(TensorDataset(X_tensor, y_tensor), batch_size=params['batch_size'], shuffle=False)

#     # Initialize the model with fixed parameters
#     model = BinaryClassifier(params['hidden_units'])
#     criterion = nn.BCELoss()
#     optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

#     # Evaluate the model
#     model.eval()
#     with torch.no_grad():
#         y_pred = model(X_tensor).numpy()
#         auc = roc_auc_score(y, y_pred)

#         # Calculate the best threshold using Youden's J statistic
#         fpr, tpr, thresholds = roc_curve(y, y_pred)
#         j_scores = tpr - fpr
#         best_threshold = thresholds[np.argmax(j_scores)]

#         final_pred_classes = (y_pred >= best_threshold).astype(int)
#         precision = precision_score(y, final_pred_classes)
#         recall = recall_score(y, final_pred_classes)
#         f1 = f1_score(y, final_pred_classes)

#         for comp, pred, actual in zip(compounds, y_pred, y):
#             all_predictions.append({
#                 'Compound': comp,
#                 'Predicted Probability': pred[0],
#                 'Actual Label': actual,
#                 'Assay': assay
#             })

#         results.append({
#             'Assay': assay,
#             'Best AUC': auc,
#             'Precision': precision,
#             'Recall': recall,
#             'F1 Score': f1
#         })
#         break

# # Output results and predictions
# results_df = pd.DataFrame(results)
# predictions_df = pd.DataFrame(all_predictions)


In [74]:
# results_df

In [75]:
# best_threshold

In [76]:
# predictions_df

In [77]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, roc_curve
# import pandas as pd
# import numpy as np
# from tqdm import tqdm

# # Model definition
# class BinaryClassifier(nn.Module):
#     def __init__(self, hidden_units):
#         super(BinaryClassifier, self).__init__()
#         self.layer1 = nn.Linear(4, hidden_units[0])
#         self.layer2 = nn.Linear(hidden_units[0], hidden_units[1])
#         self.output = nn.Linear(hidden_units[1], 1)
#         self.relu = nn.ReLU()

#     def forward(self, x):
#         x = self.relu(self.layer1(x))
#         x = self.relu(self.layer2(x))
#         return torch.sigmoid(self.output(x))

# # Function to train the model
# def train_model(X, y, hidden_units, lr, batch_size, num_epochs):
#     # Scaling the data
#     print(X.columns)
#     X = X.drop(['assay', 'expected_value_fp', 'InChICode_standardised'], axis = 1)
#     scaler = StandardScaler()
#     X_scaled = scaler.fit_transform(X)

#     # Convert to tensors
#     X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
#     y_tensor = torch.tensor(y.to_numpy(), dtype=torch.float32).view(-1, 1)

#     # DataLoader
#     loader = DataLoader(TensorDataset(X_tensor, y_tensor), batch_size=batch_size, shuffle=True)

#     # Model initialization
#     model = BinaryClassifier(hidden_units)
#     criterion = nn.BCELoss()
#     optimizer = optim.Adam(model.parameters(), lr=lr)

#     # Training loop
#     for epoch in range(num_epochs):
#         for inputs, labels in loader:
#             optimizer.zero_grad()
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()

#     return model, scaler

# # Function to test the model
# def test_model(model, scaler, X, y):
#     # Scaling and tensor conversion
#     X_scaled = scaler.transform(X)
#     X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
#     y_tensor = torch.tensor(y.to_numpy(), dtype=torch.float32).view(-1, 1)

#     # Evaluate model
#     model.eval()
#     with torch.no_grad():
#         outputs = model(X_tensor)
#         predictions = outputs.numpy()
#         auc = roc_auc_score(y, predictions)

#     return predictions, auc

# # Function to evaluate on the full dataset
# def evaluate_full_dataset(model, scaler, X, y):
#     return test_model(model, scaler, X, y)

# # Example usage
# data = df1[df1['assay'] == '588458']  # Load your data
# # features = ['feature1', 'feature2', 'feature3', 'feature4']  # Define your feature columns
# X = data[features]
# y = data['expected_value_fp']
# print(X)
# # Train the model
# model, scaler = train_model(X, y, (32, 16), 0.001, 64, 50)

# # Test the model
# predictions, auc = test_model(model, scaler, X, y)
# print(f"Test AUC: {auc}")

# # Evaluate on the full dataset
# full_predictions, full_auc = evaluate_full_dataset(model, scaler, X, y)
# print(f"Full dataset AUC: {full_auc}")
